In [47]:
import json

'''
The plan is to extract the clean fuel values from fuels_data.
Then, we'll add a new property called "value" to the "property" dict for each country in the countries dictionary.
'''

# Clean Fuels Data by Country from Who
# source: http://apps.who.int/gho/data/node.main.SDGFUELS712?lang=en
with open('SDGPOLLUTINGFUELS.json', 'r') as f:
    fuels_data = json.load(f)
    
# geojson of countries
# source: https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json
with open('countries.geo.json', 'r') as f:
    countries = json.load(f)
    
# new dictionary limited to just information we need
fuels_relevant = fuels_data['fact']

# instantiate new dict to store countries and their values
# values represent % of population with primary reliance on clean fuels
values = {}

In [48]:
# Let's take a look at our data
print('fuel data keys:', list(fuels_data.keys()))
print('country keys:', list(countries.keys()))
print()
print('Relevant Fuel Data Example (fuels_data[\'fact\']) \n', take(1, fuels_relevant), '\n')
print('Country Data Example (countries[\'features\']) \n', take(1, countries['features']), '\n')

fuel data keys: ['dimension', 'fact']
country keys: ['type', 'features']

Relevant Fuel Data Example (fuels_data['fact']) 
 [{'Value': '79', 'dim': {'PUBLISHSTATE': 'Published', 'GHO': 'Proportion of population with primary reliance on clean fuels (%)', 'COUNTRY': 'Bolivia (Plurinational State of)', 'REGION': 'Americas', 'YEAR': '2014'}}] 

Country Data Example (countries['features']) 
 [{'properties': {'name': 'Afghanistan'}, 'type': 'Feature', 'id': 'AFG', 'geometry': {'type': 'Polygon', 'coordinates': [[[61.210817, 35.650072], [62.230651, 35.270664], [62.984662, 35.404041], [63.193538, 35.857166], [63.982896, 36.007957], [64.546479, 36.312073], [64.746105, 37.111818], [65.588948, 37.305217], [65.745631, 37.661164], [66.217385, 37.39379], [66.518607, 37.362784], [67.075782, 37.356144], [67.83, 37.144994], [68.135562, 37.023115], [68.859446, 37.344336], [69.196273, 37.151144], [69.518785, 37.608997], [70.116578, 37.588223], [70.270574, 37.735165], [70.376304, 38.138396], [70.806821, 3

In [49]:
# populating our new values dictionary. removing <> signs.
for country in fuels_relevant:
    name = country['dim']['COUNTRY']
    value = country['Value']
    if value == '>95':
        value = '95'
    if value == '<5':
        value = '5'
    values[name] = value

In [50]:
# a few examples from our values dictionary
take(10, values.items())

[('Comoros', '7'),
 ('Bahamas', '95'),
 ('Iraq', '95'),
 ('Costa Rica', '95'),
 ('El Salvador', '83'),
 ('Guatemala', '36'),
 ('Sweden', '95'),
 ('Ghana', '21'),
 ('Lebanon', '95'),
 ('Mozambique', '5')]

In [51]:
# Adding a field called 'value' to each country's 'properties' dictionary.
# note: the country names in each data set don't exactly match up, so we're
# keeping track of those in 'fails' and will address them later
fails = []
for item in countries['features']:
    name = item['properties']['name']
    try:
        item['properties']['value'] = values[name]
    except:
        fails.append(name)

In [52]:
# the countries with no matches
fails

['Antarctica',
 'French Southern and Antarctic Lands',
 'The Bahamas',
 'Bermuda',
 'Bolivia',
 'Brunei',
 'Ivory Coast',
 'Republic of the Congo',
 'Northern Cyprus',
 'Falkland Islands',
 'United Kingdom',
 'Guinea Bissau',
 'Greenland',
 'French Guiana',
 'Iran',
 'South Korea',
 'Kosovo',
 'Laos',
 'Libya',
 'Moldova',
 'Macedonia',
 'New Caledonia',
 'Puerto Rico',
 'North Korea',
 'Russia',
 'Western Sahara',
 'Somaliland',
 'Republic of Serbia',
 'Syria',
 'East Timor',
 'Turkey',
 'Taiwan',
 'Venezuela',
 'Vietnam',
 'West Bank']

In [54]:
# Let's create a new dictionary of possible matches.
possibilities = {}
for fail in fails:
    for country in list(values.keys()):
        if fail in country:
            possibilities[fail] = country
possibilities

{'Bolivia': 'Bolivia (Plurinational State of)',
 'Brunei': 'Brunei Darussalam',
 'Iran': 'Iran (Islamic Republic of)',
 'Macedonia': 'The former Yugoslav republic of Macedonia',
 'Moldova': 'Republic of Moldova',
 'Republic of the Congo': 'Democratic Republic of the Congo',
 'Russia': 'Russian Federation',
 'Syria': 'Syrian Arab Republic',
 'United Kingdom': 'United Kingdom of Great Britain and Northern Ireland',
 'Venezuela': 'Venezuela (Bolivarian Republic of)'}

In [55]:
# Good! There's only one match per country. Now we can go through and look up the correct country name to assign the right value.
for country in possibilities.keys():
    key = possibilities[country]
    for item in countries['features']:
        name = item['properties']['name']
        if name == country:
            item['properties']['value'] = values[key]

In [49]:
# Finally, let's write it all to a new geojson file
with open('cleanfuels.geojson', 'w') as outfile:
    json.dump(countries, outfile)